In [2]:
import pandas as pd
import pickle
from collections import Counter

import xgboost as xgb
import seaborn as sns


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix



In [3]:

pd.set_option('display.max_rows', 300)

survey = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/Classification-Project-DSjobs/Data/kaggle-survey-2017/multi_r_.csv', encoding = 'ISO-8859-1')

list(survey.columns)

survey.JobSatisfaction.value_counts()



survey.head()

survey.EmploymentStatus.unique()

survey.groupby('EmploymentStatus').count()

survey.UniversityImportance.value_counts()

survey.JobSatisfaction.value_counts()



/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (32,84,87,88,100,110,117,124,125,128,130,131,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


7                          1448
8                          1427
6                           765
9                           677
5                           627
10 - Highly Satisfied       589
3                           358
4                           354
1 - Highly Dissatisfied     167
I prefer not to share       148
2                           117
Name: JobSatisfaction, dtype: int64

In [4]:
conditions = [  survey.JobSatisfaction.isin(['7','8','9','10 - Highly Satisfied']),
                survey.JobSatisfaction.isin(['1 - Highly Dissatisfied','2','3','4','5','6','I prefer not to share'])
             ]
choices = [1,0]
survey['JobSatisfaction'] = np.select(conditions, choices, default=survey.JobSatisfaction)


survey = survey[survey.JobSatisfaction.notna()]


In [5]:
survey.shape

(6677, 229)

In [6]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
#missing_value_df

In [7]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
seventy_five_data = missing_value_df[missing_value_df.percent_missing<40]

seventy_five_data

survey_minimal_missing= survey[list(seventy_five_data.columnname)]

survey_minimal_missing.reset_index(inplace=True)
X = survey_minimal_missing.drop(columns = 'JobSatisfaction',axis =1)
y = survey_minimal_missing['JobSatisfaction']
# X

In [8]:
Counter(y)

Counter({0: 2536, 1: 4141})

In [9]:
X_cat = X.select_dtypes(include='object')
X_cat = pd.DataFrame(X_cat,dtype ='str')
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_cat_imputed = imp_freq.fit_transform(X_cat)
X_cat_imputed = pd.DataFrame(data = X_cat_imputed, columns = X_cat.columns)

from collections import defaultdict
d = defaultdict(LabelEncoder)
fit = X_cat_imputed.apply(lambda x: d[x.name].fit_transform(x))
#fit

In [10]:
X_num = X.select_dtypes(include='float64')
imp_freq = SimpleImputer(missing_values=np.nan, strategy='median')
X_num_imputed = imp_freq.fit_transform(X_num)
X_num_imputed = pd.DataFrame(data= X_num_imputed, columns = X_num.columns)

scale = StandardScaler()
X_num_imputed = scale.fit_transform(X_num_imputed)
X_num_imputed = pd.DataFrame(data= X_num_imputed, columns = X_num.columns)



In [11]:
# for i in range(X_cat_imputed.shape[1]):
#     print(X_cat.iloc[:,i].value_counts())


In [12]:
X_imputed = pd.concat([fit,X_num_imputed],axis =1)
colnames = X_imputed.columns

In [13]:
y =pd.Series(y, dtype ='int')

In [14]:
Counter(y)

Counter({0: 2536, 1: 4141})

In [15]:
# X_imputed.sample(20)

In [16]:
X, X_test, y, y_test = train_test_split(X_imputed, y, test_size=.2, random_state=100)






In [17]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier()
gbm.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [18]:
gbm.score(X_test,y_test)

0.68413173652694614

In [19]:
Counter((gbm.predict(X_test)))

Counter({1: 1027, 0: 309})

In [20]:
# pd.DataFrame(zip(list(gbm.feature_importances_),colnames)).sort_values(by = 0, ascending= False)

In [28]:
parameter_grid={
    "loss":["deviance"],
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "learning_rate": [0.075],
    "min_samples_split": np.linspace(0.01, 0.1, 4),
    "min_samples_leaf": np.linspace(0.1, 0.5, 4),
    "max_depth":[7,10],
    "max_features":["sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[ 1.0],
    "n_estimators":[125,130,135]
    }
clf = GridSearchCV(GradientBoostingClassifier(), parameter_grid, cv=5, n_jobs=-1,scoring = 'precision', verbose= True)

clf.fit(X, y)



Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 19.1min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 25.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'loss': ['deviance'], 'learning_rate': [0.075], 'min_samples_split': array([ 0.01,  0.04,  0.07,  0.1 ]), 'min_samples_leaf': array([ 0.1    ,  0.23333,  0.36667,  0.5    ]), 'max_depth': [7, 10], 'max_features': ['sqrt'], 'criterion': ['friedman_mse', 'mae'], 'subsample': [1.0], 'n_estimators': [75, 100, 125]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='precision', verbose=True)

In [29]:
pickle.dump(clf,open('gbmCV.p','wb'))

In [30]:
clf = pickle.load(open('gbmCV.p','rb'))

In [31]:
clf.best_score_

0.70324802217613569

In [32]:
clf.best_params_


{'criterion': 'friedman_mse',
 'learning_rate': 0.075,
 'loss': 'deviance',
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 0.10000000000000001,
 'min_samples_split': 0.040000000000000001,
 'n_estimators': 125,
 'subsample': 1.0}

In [ ]:
confusion_matrix(y_test,clf.predict(X_test))

In [ ]:
Counter(clf.predict(X_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

Run CV with 5 folds (Random Forest)

Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [6, 15,20, 30,40]
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 3],
    'n_estimators': [200, 400]
}

rf = RandomForestClassifier()
rf_grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=10, n_jobs=-1, scoring ='percision')
rf_grid.fit(X, y)

In [ ]:
rf_grid.best_score_

In [ ]:
pickle.dump(rf_grid,open('rfCV.p','wb'))

In [ ]:
rf_grid = pickle.load(open('rfCV.p','rb'))

In [ ]:
confusion_matrix(y_test,rf_grid.predict(X_test))